In [ ]:
# old
import pandas as pd
import numpy as np

def load_dfs(file_path1, file_path2, dtype=None):
    df1 = pd.read_csv(file_path1, dtype=dtype)
    df2 = pd.read_csv(file_path2, dtype=dtype)
    return df1, df2

def simple_compare_one_column(df1, df2, coluna):
    # Passo 1: Garantir que ambos os DataFrames estão ordenados pela coluna especificada
    df1_sorted = df1.sort_values(by=coluna).reset_index(drop=True)
    df2_sorted = df2.sort_values(by=coluna).reset_index(drop=True)

    # Passo 2: Comparar as colunas especificadas e criar uma nova coluna de verificação
    df_comparado = pd.DataFrame({
        f'{coluna}_df1': df1_sorted[coluna],
        f'{coluna}_df2': df2_sorted[coluna],
        'verificacao': df1_sorted[coluna] == df2_sorted[coluna]
    })

    # Passo 3: Adicionar colunas n_true e n_false
    df_comparado['n_true'] = df_comparado['verificacao'].sum()  # Conta o número de True
    df_comparado['n_false'] = len(df_comparado) - df_comparado['n_true']  # Conta o número de False

    return df_comparado
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------
# 
#   Cara usa esses dois aqui em baixo, eles verificam de verdade as coisas
#------------------------------------------------------------------------------------------------
def criar_dataframe_comparativo(df1, df2, col_chave, col_compare):
    # Ordenando os DataFrames pela coluna col_chave
    df1_sorted = df1.sort_values(by=col_chave).reset_index(drop=True)
    df2_sorted = df2.sort_values(by=col_chave).reset_index(drop=True)
    # Criando o novo DataFrame comparativo
    df_comparativo = pd.DataFrame({
        f"{col_chave}_df1": df1_sorted[col_chave],
        f"{col_chave}_df2": df2_sorted[col_chave],
        f"{col_compare}_df1": df1_sorted[col_compare],
        f"{col_compare}_df2": df2_sorted[col_compare]
    })
    
    return df_comparativo
#------------------------------------------------------------------------------------------------
def calcule_diferenca(df1, df2, col_chave, col_comparar):
    df_comparativo = criar_dataframe_comparativo(df1, df2, col_chave, col_comparar)
    df_comparativo[f'{col_chave}_igual'] = df_comparativo.apply(lambda row: 1 if row[f'{col_chave}_df1'] == row[f'{col_chave}_df2'] else 0, axis=1)
    df_comparativo[f'{col_comparar}_igual'] = df_comparativo.apply(lambda row: 1 if row[f'{col_comparar}_df1'] == row[f'{col_comparar}_df2'] else 0, axis=1)
    df_comparativo[f'soma_total_{col_chave}_igual'] = df_comparativo[f'{col_chave}_igual'].sum()
    df_comparativo[f'soma_total_{col_comparar}_igual'] = df_comparativo[f'{col_comparar}_igual'].sum()
    return df_comparativo
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------
# def load_and_print_shape(df1, df2):
    # df1 = pd.read_csv(file_path1)
    # df2 = pd.read_csv(file_path2)
def look_shapes_and_types(df1, df2):
    print(f"Shape of DataFrame 1: {df1.shape}")
    print(f"Columns of DataFrame 1: {df1.columns}")
    print(f"Shape of DataFrame 2: {df2.shape}")
    print(f"Columns of DataFrame 2: {df2.columns}")
    
    if df1.shape[0] != df2.shape[0]:
        print('ERRO !')
        print('DataFrames have different number of rows')
    if df1.shape[1] != df2.shape[1]:
        print('ERRO !')
        print('DataFrames have different number of columns')
    
    # Checar tipos de colunas diferentes
    z=[]
    for col in df1.columns:
        if col in df2.columns:
            dtype1 = df1[col].dtype
            dtype2 = df2[col].dtype
            if dtype1 != dtype2:
                z.append(col)
                print(f"Type mismatch in column '{col}':")
                print(f" - DataFrame 1: {dtype1}")
                print(f" - DataFrame 2: {dtype2}")            
        else:
            print(f"Column '{col}' is missing in DataFrame 2")
    if len(z) == 0:
        print('*'*50)
        print('All columns have the same data types')
        print('*'*50)
    for col in df2.columns:
        if col not in df1.columns:
            print(f"Column '{col}' is missing in DataFrame 1")
    
    # return df1, df2
# def load_and_print_shape(file_path1, file_path2):
#     df1 = pd.read_csv(file_path1)
#     df2 = pd.read_csv(file_path2)
#     print(f"Shape of DataFrame 1: {df1.shape}")
#     print(f"Shape of DataFrame 1: {df1.columns}")
#     print(f"Shape of DataFrame 2: {df2.shape}")
#     print(f"Shape of DataFrame 2: {df2.columns}")
#     if df1.shape[0] != df2.shape[0]:
#         print('ERRO !')
#         print('DataFrames have different number of rows')
#     if df1.shape[1] != df2.shape[1]:
#         print('ERRO !')
#         print('DataFrames have different number of columns')
#     return df1, df2

def print_markdown_table(df):
    # Mostra o cabeçalho da tabela
    header = "| " + " | ".join(df.columns) + " |"
    separator = "| " + " | ".join(["---"] * len(df.columns)) + " |"

    # Mostra as linhas do DataFrame
    rows = "\n".join(
        ["| " + " | ".join(map(str, row)) + " |" for row in df.values]
    )

    # Exibe o resultado em formato de tabela Markdown
    markdown_table = header + "\n" + separator + "\n" + rows
    print(markdown_table)

def compare_group_sizes(df1, df2, group_column='CD_PESS'):
    # Agrupando por 'group_column' e obtendo o tamanho dos grupos
    # group_df1 = df1.groupby(group_column).size().sort_values(ascending=False)
    # group_df2 = df2.groupby(group_column).size().sort_values(ascending=False)

    group_df1 = df1.round({group_column: 7}).groupby(group_column).size().sort_values(ascending=False)
    group_df2 = df2.round({group_column: 7}).groupby(group_column).size().sort_values(ascending=False)


    # Criando um DataFrame para armazenar as colunas
    result = pd.DataFrame({
        'df1_size': group_df1,
        'df2_size': group_df2
    })

    # Preenchendo valores ausentes com 0 (caso um código exista em apenas um dos DataFrames)
    result = result.fillna(0)

    # Adicionando uma coluna com a subtração dos tamanhos
    result['size_difference'] = result['df1_size'] - result['df2_size']
    diferente = result[result['size_difference'] != 0]
        
    return result, diferente

def print_all(df1, df2):
    print('+'*50)
    print(' COMPARAÇÕES DE TAMANHO DOS GRUPOS ')
    print('+'*50)
    i = 0
    for col in df1.columns:
        r, d = compare_group_sizes(df1, df2, group_column=col)
        print(f' COLUNA : {col}')
        if d.shape[0] != 0:
            i+=1
            print(' ERRADO '*5)
            df1_group = df1.groupby(col).size().sort_values(ascending=False)#.isin(d.index)]
            df2_group = df2.groupby(col).size().sort_values(ascending=False)#.isin(d.index)]
            # print(f"Shape of DataFrame 1: {df1_group.shape}")
            # print(f"Shape of DataFrame 2: {df2_group.shape}")
            print(f' shape df1.{col} = {df1_group.shape[0]}')
            print(df1_group.head(5))
            print(f' shape df2.{col} = {df2_group.shape[0]}')
            print(df2_group.head(5))

            print('d.shape[0] = ', d.shape[0])
            print('diferences:')
            print(d.head(4))
            print(d.tail(4))
            # print_markdown_table(d)
            print('+'*50)
        # else:
            
            # print(' CERTO ')
            # print('r.shape[0] = ', r.shape[0])
            # print(r.head(5))
            # print('+'*50)
    if i == 0:
        print(' Tudo CERTO ')
    elif i != 0:
        print(' ERRADO '*5)
        print(f' Tem {len(df1.columns)} colunas e {i} contém diferenças')
def run_path(path1, path2):
    df1, df2 = load_dfs(path1, path2)
    print(df1.info())
    print(df2.info())
    print_all(df1, df2)
def run(df1, df2):
    print(df1.info())
    print(df2.info())
    print_all(df1, df2)

# def result_of_compare_column_elements(file_path1, file_path2, column_name):
def result_of_compare_column_elements(df1, df2, column_name):
    # Obter os valores únicos da coluna em ambos os DataFrames e arredondar para 8 casas decimais
    if pd.api.types.is_numeric_dtype(df1[column_name]):
        # df1_elements = set(round(val, 5) for val in df1[column_name])
        df1_elements = set(val for val in df1[column_name])
    else:
        # Se não for numérica, apenas converter para conjunto sem arredondamento
        df1_elements = set(df1[column_name])
    if pd.api.types.is_numeric_dtype(df2[column_name]):
        # df2_elements = set(round(val, 5) for val in df2[column_name])
        df2_elements = set(val for val in df2[column_name])# setiver nan
    else:
        # Se não for numérica, apenas converter para conjunto sem arredondamento
        df2_elements = set(df2[column_name])
    
    # df1_elements = set(round(val, 5) for val in df1[column_name])
    # df2_elements = set(round(val, 5) for val in df2[column_name])
    
    # Identificar os elementos que estão somente em df1 e somente em df2
    only_in_df1 = list(df1_elements - df2_elements)
    only_in_df2 = list(df2_elements - df1_elements)
    
    # Verificar se os tamanhos das listas são diferentes
    if len(only_in_df1) != len(only_in_df2):
        print(f"Tamanho de 'only_in_df1' {column_name}: {len(only_in_df1)}")
        print(f"Tamanho de 'only_in_df2' {column_name}: {len(only_in_df2)}")
        
        # Garantir que ambas as listas tenham o mesmo tamanho
        max_len = max(len(only_in_df1), len(only_in_df2))
        only_in_df1.extend([np.nan] * (max_len - len(only_in_df1)))
        only_in_df2.extend([np.nan] * (max_len - len(only_in_df2)))
    
    # Criar o DataFrame de resultado com duas colunas
    result_df = pd.DataFrame({
        f'Only_in_df1_{column_name}': only_in_df1,
        f'Only_in_df2_{column_name}': only_in_df2
    })
    
    return result_df


def compare_dataframes(df1, df2):
    import pdb; pdb.set_trace()
    # Verificar se ambos os DataFrames têm o mesmo número de linhas e colunas
    if df1.shape != df2.shape:
        print(f"ERRO! Os DataFrames têm tamanhos diferentes: {df1.shape} vs {df2.shape}")
        return None

    # Verificar se ambos os DataFrames têm as mesmas colunas na mesma ordem
    if not df1.columns.equals(df2.columns):
        print("ERRO! Os DataFrames têm colunas diferentes ou em ordens diferentes.")
        return None

    # Verificar a igualdade dos valores em cada célula
    comparison = df1.equals(df2)

    if comparison:
        print("Os DataFrames são iguais em todos os valores.")
    else:
        print("Diferenças encontradas:")
        # Localizar e imprimir as diferenças
        differences = df1.ne(df2)
        diff_locations = differences.stack()[differences.stack()]
        for index, value in diff_locations.items():
            print(f"Diferença encontrada em {index}: df1={df1.loc[index]}, df2={df2.loc[index]}")

    return comparison

# path1 = 'comparacoes/df_geral_s23.csv'
# path2 = 'comparacoes/df_geral_s23.csv'
# df1, df2 = load_and_print_shape(path1, path2)

In [ ]:
path1 = 'comparisson/MAZOQUE.csv'
path2 = 'comparisson/MAZOQUE.csv'


df1, df2 = load_dfs(path1, path2)
# criar_dataframe_comparativo(df1, df2, 'col_chave', 'col_to_compare')
look_shapes_and_types(df1, df2)
# calcule_diferenca(df1, df2, 'col_chave', 'col_to_compare').head(4)
# result_of_compare_column_elements(df1, df2, 'col_to_compare').to_csv('comparisson/result_cabeca_chave_CD_PESS.csv', index=False)
# simple_compare_one_column(df1, df2, 'col_to_compare')
run(df1, df2)

olds

In [ ]:
def compare_group_sizes_and_rows(df1, df2, group_column='CD_PESS'):
    # Ordenando os DataFrames por group_column antes de comparar
    df1_sorted = df1.sort_values(by=group_column).reset_index(drop=True)
    df2_sorted = df2.sort_values(by=group_column).reset_index(drop=True)

    # Comparando se as linhas (após ordenação) são iguais
    rows_equal = df1_sorted.equals(df2_sorted)

    # Agrupando por 'group_column' e obtendo o tamanho dos grupos
    group_df1 = df1.round({group_column: 7}).groupby(group_column).size().sort_values(ascending=False)
    group_df2 = df2.round({group_column: 7}).groupby(group_column).size().sort_values(ascending=False)

    # Criando um DataFrame para armazenar as colunas
    result = pd.DataFrame({
        'df1_size': group_df1,
        'df2_size': group_df2
    })

    # Preenchendo valores ausentes com 0 (caso um código exista em apenas um dos DataFrames)
    result = result.fillna(0)

    # Adicionando uma coluna com a subtração dos tamanhos
    result['size_difference'] = result['df1_size'] - result['df2_size']
    diferente = result[result['size_difference'] != 0]

    return result, diferente, rows_equal

def print_all_with_row_comparison(df1, df2):
    print('+'*50)
    print(' COMPARAÇÕES DE TAMANHO DOS GRUPOS E LINHAS ')
    print('+'*50)
    i = 0
    for col in df1.columns:
        r, d, rows_equal = compare_group_sizes_and_rows(df1, df2, group_column=col)
        print(f' COLUNA : {col}')
        if d.shape[0] != 0:
            i += 1
            print(' ERRADO '*5)
            df1_group = df1.groupby(col).size().sort_values(ascending=False)
            df2_group = df2.groupby(col).size().sort_values(ascending=False)
            print(f' shape df1.{col} = {df1_group.shape[0]}')
            print(df1_group.head(5))
            print(f' shape df2.{col} = {df2_group.shape[0]}')
            print(df2_group.head(5))

            print('d.shape[0] = ', d.shape[0])
            print('diferences:')
            print(d.head(4))
            print(d.tail(4))
            print('+'*50)
        else:
            print(' CERTO ')
            print('r.shape[0] = ', r.shape[0])
            print(r.head(5))
            print('+'*50)

        # Comparando as linhas
        if not rows_equal:
            print(' ERRADO NA ORDEM DAS LINHAS ')
        else:
            print(' LINHAS ESTÃO CORRETAS ')

    if i == 0:
        print(' Tudo CERTO ')
    elif i != 0:
        print(' ERRADO '*5)
        print(f' Tem {len(df1.columns)} colunas e {i} contém diferenças')

def compare_rows_by_key(df1, df2, key_column='CD_PTOV'):
    # Definir a chave como índice para os dois DataFrames
    df1_keyed = df1.set_index(key_column)
    df2_keyed = df2.set_index(key_column)
    
    # Identificar chaves comuns e chaves que faltam em um dos DataFrames
    common_keys = df1_keyed.index.intersection(df2_keyed.index)
    missing_in_df1 = df2_keyed.index.difference(df1_keyed.index)
    missing_in_df2 = df1_keyed.index.difference(df2_keyed.index)
    
    # Comparar as linhas para as chaves comuns
    differences = []
    for key in common_keys:
        row_df1 = df1_keyed.loc[key]
        row_df2 = df2_keyed.loc[key]
        if not row_df1.equals(row_df2):
            differences.append((key, row_df1, row_df2))
    
    return common_keys, missing_in_df1, missing_in_df2, differences

def print_comparison_by_key(df1, df2, key_column='CD_PTOV'):
    print('+'*50)
    print(f'COMPARAÇÃO BASEADA NA CHAVE: {key_column}')
    print('+'*50)

    common_keys, missing_in_df1, missing_in_df2, differences = compare_rows_by_key(df1, df2, key_column)

    if missing_in_df1.empty and missing_in_df2.empty and len(differences) == 0:
        print('Tudo CERTO: As chaves e os dados correspondem entre os DataFrames.')
    else:
        if not missing_in_df1.empty:
            print('FALTANDO FALTANDO Faltando no df1:')
            print(missing_in_df1)
            
        if not missing_in_df2.empty:
            print('FALTANDO FALTANDO Faltando no df2:')
            print(missing_in_df2)
        if differences:
            print('DIFERENÇAS ENCONTRADAS DIFERENÇAS ENCONTRADAS:')
            for key, row_df1, row_df2 in differences:
                print(f'Chave: {key}')
                print(f'Dados no df1: \n{row_df1}')
                print(f'Dados no df2: \n{row_df2}')
                print('-'*50)

def compare_rows_by_key_old(df1, df2, key_column='CD_PTOV'):
    # Definir a chave como índice para os dois DataFrames
    df1_keyed = df1.set_index(key_column)
    df2_keyed = df2.set_index(key_column)
    
    # Identificar chaves comuns e chaves que faltam em um dos DataFrames
    common_keys = df1_keyed.index.intersection(df2_keyed.index)
    missing_in_df1 = df2_keyed.index.difference(df1_keyed.index)
    missing_in_df2 = df1_keyed.index.difference(df2_keyed.index)
    
    # Comparar as linhas para as chaves comuns
    differences = []
    for key in common_keys:
        row_df1 = df1_keyed.loc[key]
        row_df2 = df2_keyed.loc[key]
        if not row_df1.equals(row_df2):
            differences.append((key, row_df1, row_df2))
    
    return common_keys, missing_in_df1, missing_in_df2, differences

def write_comparison_to_file_old(df1, df2, key_column='CD_PTOV', file_name='comparison_output.txt'):
    with open(file_name, 'w') as file:
        file.write('+'*50 + '\n')
        file.write(f'COMPARAÇÃO BASEADA NA CHAVE: {key_column}\n')
        file.write('+'*50 + '\n')

        common_keys, missing_in_df1, missing_in_df2, differences = compare_rows_by_key_old(df1, df2, key_column)

        if missing_in_df1.empty and missing_in_df2.empty and len(differences) == 0:
            file.write('Tudo CERTO: As chaves e os dados correspondem entre os DataFrames.\n')
        else:
            if not missing_in_df1.empty:
                file.write('Faltando no df1:\n')
                file.write(missing_in_df1.to_string() + '\n')
            if not missing_in_df2.empty:
                file.write('Faltando no df2:\n')
                file.write(missing_in_df2.to_string() + '\n')
            if differences:
                file.write('Diferenças encontradas:\n')
                for key, row_df1, row_df2 in differences:
                    file.write(f'Chave: {key}\n')
                    file.write('Dados no df1:\n')
                    file.write(row_df1.to_string() + '\n')
                    file.write('Dados no df2:\n')
                    file.write(row_df2.to_string() + '\n')
                    file.write('-'*50 + '\n')

def compare_rows_by_key(df1, df2, key_column='CD_PTOV'):
    # Definir a chave como índice para os dois DataFrames
    df1_keyed = df1.set_index(key_column)
    df2_keyed = df2.set_index(key_column)
    
    # Identificar chaves comuns e chaves que faltam em um dos DataFrames
    common_keys = df1_keyed.index.intersection(df2_keyed.index)
    missing_in_df1 = df2_keyed.index.difference(df1_keyed.index)
    missing_in_df2 = df1_keyed.index.difference(df2_keyed.index)
    
    # Comparar as linhas para as chaves comuns
    differences = []
    for key in common_keys:
        row_df1 = df1_keyed.loc[key]
        row_df2 = df2_keyed.loc[key]
        if not row_df1.equals(row_df2):
            differences.append((key, row_df1, row_df2))
    
    return common_keys, missing_in_df1, missing_in_df2, differences
def write_comparison_to_file(df1, df2, key_column='CD_PTOV', file_name='comparison_output.txt'):
    with open(file_name, 'w') as file:
        file.write(f'COMPARAÇÃO BASEADA NA CHAVE: {key_column}\n')
        file.write(f'Chave;Coluna;Valor_df1;Valor_df2\n')  # Cabeçalho para o arquivo CSV-like

        common_keys, missing_in_df1, missing_in_df2, differences = compare_rows_by_key(df1, df2, key_column)

        if missing_in_df1.empty and missing_in_df2.empty and len(differences) == 0:
            file.write('Tudo CERTO: As chaves e os dados correspondem entre os DataFrames.\n')
        else:
            if not missing_in_df1.empty:
                file.write(f'Faltando no df1 (Presentes no df2): {",".join(missing_in_df1)}\n')
            if not missing_in_df2.empty:
                file.write(f'Faltando no df2 (Presentes no df1): {",".join(missing_in_df2)}\n')
            
            if differences:
                # Escrevendo diferenças em formato tabular (CSV-like)
                for key, row_df1, row_df2 in differences:
                    for column in row_df1.index:  # Iterar sobre as colunas
                        val_df1 = row_df1[column]
                        val_df2 = row_df2[column]
                        if val_df1 != val_df2:
                            file.write(f'{key};{column};{val_df1};{val_df2}\n')

def df_de_duplicados(df, col):
    df_dup_check = df.groupby(col).size().reset_index(name='count')
    df_dup_check = df_dup_check[df_dup_check['count'] > 1]
    return df_dup_check